Numpy package used to store and manipulate data

In [1]:
import numpy as np
from IPython.display import display, Math

Using a numpy array to represent qubits, fast and manipulable. where:

$$ qubit(a,b)  = \frac{a}{\sqrt{|a|^{2}+|b|^{2}}}|0\rangle+\frac{a}{\sqrt{|a|^{2}+|b|^{2}}}|1\rangle\ $$

In [2]:
def qubit(a,b):                          #function creates a normed vector with 2 entries
    Length = np.sqrt(np.abs(a)**2+np.abs(b)**2)
    return(np.array([[a/Length,b/Length]]).T)

print((qubit(1,0)))

[[1.]
 [0.]]


In [3]:
def qudit(*args):                         #function creates a normed vector with arbitrary entries
    Length = sum([i**2 for i in args])**(1/len(args))
    return(np.array([j/Length for j in args]))

Want to generate a random number uniformly between 0 and 1 to simulate measurements.  
remove 0s  
permute all quotients of pairs of coefficients, take the maximum.  
1/max determines how finely to generate.

Gates are represented using numpys matrix functions. The function below accepts four parameters for each of the entries in a 2x2 matrix and one additional coefficent parameter to scale all the individual entries by. 

$$gate(a\ ,b\ ,c\ ,d\ ,\mathit{coefficient}\ ) = \mathit{coefficient}*\begin{pmatrix} a& b\\ c&d \end{pmatrix} $$

The coefficient parameter is 1 by default and is useful for representing gates like the hadamard gate as shown below:

$$gate(1\ ,1\ ,1\ ,-1\ ,2**(-0.5)\ ) = \frac{1}{\sqrt{2}}\begin{pmatrix} 1& 1\\  1&-1 \end{pmatrix} $$

In [4]:
def gate(a ,b ,c ,d ,coefficient = 1):
    matrix = np.matrix([[a,b],[c,d]])*coefficient                            #forms the matrix
    conj_trans_matrix = (coefficient*np.matrix([[a,c],[b,d]])).conj()        #forms the conjugate transpose of the matrix
    if np.allclose(matrix*conj_trans_matrix, np.matrix([[1.,0.],[0.,1.]])):  #Checks if the matrix is unitary
        return(matrix)
    raise Exception('Coefficients do not form a unitary matrix')
    
#later: if the matrix formed is some factor off of being unitary then automatically convert it.
#np.allclose only compares each element within tolerance

Assigned variable names to some commonly used gates below for ease of use.

Also serves to test the 'gate' function defined above, no errors raised means all the gates are unitary as expected

In [5]:
##Some commonly used gates are initialised below
I     = gate(1,0,0,1)            #The identity gate
Hgate = gate(1,1,1,-1,2**(-0.5)) # Hadamard gate
Xgate = gate(0,1,1,0)           # Pauli-X gate
Ygate = gate(0,-1j,1j,0)         # Pauli-Y gate
Zgate = gate(1,0,0,-1)          # Pauli-Z gate
Sgate = gate(1,0,0,1j)          # Phase gate
Tgate = gate(1,0,0,1j**0.5)     # Pi/8 gate

print(np.allclose(Xgate*Ygate*Xgate,-Ygate)) # verification that XYX = -Y

True


Naturally numpy allows multiplication of vectors and matrices easily and one can see below that after applying the Hadamard gate to the initial state:

$$1|0\rangle+0|1\rangle$$ 
one obtains the superposition:

$$\frac{1}{\sqrt{2}}|0\rangle+\frac{1}{\sqrt{2}}|1\rangle\ $$

In [6]:
print(Hgate*qubit(1,0))

[[0.70710678]
 [0.70710678]]


In [7]:
def Cgate(gate):          #forms a controlled gate(4x4) from a regular gate(2x2).
    row1 = [1,0,0,0]
    row2 = [0,1,0,0]
    row3 = [0,0,gate[0,0],gate[0,1]]
    row4 = [0,0,gate[1,0],gate[1,1]]
    return(np.matrix([row1,row2,row3,row4]))

# Deutsch's algorithm

## Step 1

first prepare the input state: $|\psi_0\rangle = |01\rangle = (1,0) \otimes (0,1)$

In [8]:
Psi0 = np.kron(qubit(1,0),qubit(0,1))
display(Math(r'|\psi_0\rangle = ({}, {}, {}, {})'.format(*[round(float(i),8) for i in Psi0])))

<IPython.core.display.Math object>

## Step 2
Apply the Hadamard gate to each qubit individually  
$ |\psi_1\rangle = ( H\otimes H)\cdot|\psi_0\rangle  $

In [9]:
Psi1 = np.kron(Hgate,Hgate)*Psi0
display(Math(r'|\psi_1\rangle = ({}, {}, {}, {})'.format(*[round(float(i),8) for i in Psi1])))

<IPython.core.display.Math object>

## Step 3
Construct the gate $U_f$ and apply it to $|\psi_1\rangle$

$ |\psi_2 \rangle = U_f \cdot |\psi_1\rangle $

In [10]:
def f_1(x):
    return(x)

def f_2(x):
    return(1-x)

def f_3(x):
    return(0)

def f_4(x):
    return(1)

def U_f(function):
    a = [1,0,0,0]
    b = [0,1,0,0]
    c = [0,0,1,0]
    d = [0,0,0,1]
    if function(0) == 1:
        a,b = b,a
    if function(1) == 1:
        c,d = d,c
    return(np.matrix([a,b,c,d]))

In [11]:
Psi2 = U_f(f_2)*Psi1
display(Math(r'|\psi_2\rangle = ({}, {}, {}, {})'.format(*[round(float(i),8) for i in Psi2])))

<IPython.core.display.Math object>

## Step 4
Apply the Hadamard gate to the first qubit only:  
$ |\psi_3\rangle = ( H\otimes I)\cdot|\psi_2\rangle$

In [12]:
Psi3 = np.kron(Hgate,I)*Psi2

display(Math(r'|\psi_3\rangle = ({}, {}, {}, {})'.format(*[round(float(i),8) for i in Psi3])))

<IPython.core.display.Math object>

In [13]:
print('The probability of the first qubit being 0 is:', int(Psi3[0]**2+Psi3[1]**2))
print('The probability of the first qubit being 1 is:', int(Psi3[2]**2+Psi3[3]**2)) #redundant, here for testing

The probability of the first qubit being 0 is: 0
The probability of the first qubit being 1 is: 1


# Conclusion

There are four functions from the set {0,1} to itself as follows:  
$f_1(x)=x$  
$f_2(x)=1-x$  
$f_3(x)=0$  
$f_4(x)=1$  

In [14]:
for i in (f_1,f_2,f_3,f_4):
    Finalstate = np.kron(Hgate,I)*U_f(i)*np.kron(Hgate,Hgate)*np.kron(qubit(1,0),qubit(0,1))
    prob0 = int(Finalstate[0]**2+Finalstate[1]**2)
    prob1 = int(Finalstate[2]**2+Finalstate[3]**2)#prob1 = 1 - prob0
    #print('for the gate f1 the first qubit ')
    display(Math('\\text{For the gate constructed from the function }' + i.__name__ + '\\text{ the final state of the first qubit is}'
    + r' \ {}|0\rangle + {}|1\rangle '.format( prob0, prob1)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

***
One can see from the results above that for the functions $f_1$ and $f_2$ where $f(0) \neq f(1) $ the first qubit of the output is always $|1\rangle$ .  

For the constant functions $f_3$ and $f_4$ where $f(0) = f(1)$ the first qubit of the output is always in the state $|0\rangle$

# Deutsch-Jozsa algorithm

## Step 1

Begin by preparing the state $|\psi_0 \rangle $ with $n$ qubits set to the state $|0 \rangle$ and one final qubit in the state $|1\rangle$

$|\psi_0 \rangle = |0\rangle ^{\otimes n} \otimes | 1 \rangle$

In [15]:
n = 6 #number of qubits to simulate

def all_zero(n):                        #prepares n qubits |0⟩ 
    Psi = qubit(1,0)
    for i in range(n-1):
        Psi = np.kron(Psi,qubit(1,0))
    return(Psi)

Psi_0 = np.kron(all_zero(n),qubit(0,1)) #adds the final qubit |1⟩ 

## Step 2
The Hadamard transform is next applied to $|\psi_0 \rangle$

$|\psi_1 \rangle = H^{\otimes n+1}\mid \psi_0 \rangle$

In [16]:
def Hadamard_transform(n): #Kronecker product of n Hadamard gates. 2^(n+1) square matrix
    Hform = Hgate
    for i in range(n):
        Hform = np.kron(Hgate,Hform)
    return(Hform)

Psi_1 = Hadamard_transform(n)*Psi_0

## Step 3

The gate $U_f$ is prepared and applied to $\mid \psi_1 \rangle$ 

$\mid \psi_2 \rangle = U_f \mid \psi_1 \rangle $

In [17]:
def f_bal(x,n):                     #left half 0s, right half 1s
    return(round(x/(2**n-1)))

def f_bala(x,n=0):                  # x modulo 2
    return(x%2)

def f_cons0(x,n=0):
    return(0)

def f_cons1(x,n=0):
    return(1)

def U_fn(function,n):
    Gate = np.identity(2**(n+1))                                    #create a 2^(n+1) square identity matrix
    for i in range(2**n):                                           #split the matrix into pairs of rows, indexed from 0 to 2^n
        if function(i,n) == 1:                                      #compute f(index) of the pair being tested.
            Gate[[2*i]],Gate[[2*i+1]] = Gate[[2*i+1]],Gate[[2*i]]   #If f(index) = 1 then swap the pair
    return(Gate)

In [18]:
Psi_2 = U_fn(f_cons0, n)*Psi_1

## Step 4

Apply the Hadamard transform to the qubits in the query register

$\mid \psi_3 \rangle = (H^{\otimes n}\otimes I_2) \cdot \mid \psi_2 \rangle $

In [19]:
Psi_3 = np.kron(Hadamard_transform(n-1),I)*Psi_2

## Step 5
measurement

In [20]:
def DJ_test(statevector): #compute the sum of the squared magnitudes of the first two elements of the statevector
    if round(float(statevector[0])**2 + float(statevector[1])**2,10): 
        return('is a constant function') #function is constant if the sum is 1
    return('is a balanced function')     #function is balanced if the sum is 0

In [21]:
Query = []
for i in range(2**n):
    Query += [round( float(Psi_3[2*i])**2 + float(Psi_3[2*i+1])**2 ,10)]
print(len(Query))
#print(Query)

64


In [22]:
functions = [f_bal, f_bala, f_cons0, f_cons1]
n = 7
for i in functions:
    state = np.kron(Hadamard_transform(n-1),I)*U_fn(i, n)*Hadamard_transform(n)*np.kron(all_zero(n),qubit(0,1))
    print(i.__name__ , DJ_test(state))

f_bal is a balanced function
f_bala is a balanced function
f_cons0 is a constant function
f_cons1 is a constant function


Handling numerical imprecision  
Expense of creating $U_f$